In [56]:
import numpy as np
import pandas as pd
import sympy

import torch
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Grayscale

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score

plt.style.use("../mpl styles/dracula.mplstyle")

In [10]:
class MyDataSet(Dataset):
    def __init__(self, length, transform=None):
        self.X = torch.ones(length, 2)
        self.y = torch.ones(length, 1)
        self.length = length
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.X[index], self.y[index]
        if self.transform is not None:
            return self.transform(sample)
        return sample
    
    def __len__(self):
        return self.length

In [63]:
arr = np.random.randint(1, 10, (10, 10, 3))

preprocess = Compose([
    ToTensor(),
    Resize((5, 5)),
    Grayscale(3),
])
preprocess

Compose(
    ToTensor()
    Resize(size=(5, 5), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
)

In [64]:
preprocess(arr).shape

torch.Size([3, 5, 5])